In [213]:
%load_ext autoreload
%autoreload 2
import os, sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from dataset import *
from models import *
from utils import *

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [192]:
# Load and preprocess data:
path = 'stocks/individual_stocks_5yr/'
stocks = os.listdir(path)
# Load file, pick only open, high, low, close, and convert to numpy array:
ts = pd.read_csv(path+stocks[100]).select_dtypes(['float']).values
# Normalize data to range [-1, 1]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ts_norm = scaler.fit_transform(ts)

In [193]:
# Setup device:
device = torch.device('cuda') if torch.cuda.is_available() else torch.cpu()

In [194]:
# Get train loader and test loader:
test_size = 0.2
source_len = 90
target_len = 10
batch_size = 32
trainset, testset, trainloader, testloader = get_datasets_loaders(ts_norm, ts_norm, test_size, source_len, target_len, batch_size)

In [217]:
# Hyper-parameters:
source_size = 4
target_size = 4
hidden_size = 256
num_layers = 2
dropout = 0.2
bidirect = True
teacher_forcing_ratio = 0.5

epochs = 30
lr = 1e-3

total_steps = np.ceil(len(trainset) / batch_size)
print_every = 5

In [282]:
# Initialize model:
encoder = Encoder(source_size, hidden_size, num_layers, dropout, bidirect)
decoder = Decoder(target_size, hidden_size, num_layers, dropout)
encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)

# Training loop:
for epoch in range(epochs):
    for i, (src, tgt) in enumerate(trainloader):
        encoder.train(); decoder.train()
        optimizer.zero_grad()
        src, tgt = src.to(device), tgt.to(device)
        _, hidden = encoder(src)
        outputs, _ = decoder(hidden, len(tgt[0]), tgt, torch.rand(1) >= teacher_forcing_ratio)
        loss = F.mse_loss(outputs, tgt)
        loss.backward()
        optimizer.step()
        
        # Print out training progress (on same line):
        stats = '[%d/%d][%d/%d]\tTrain MSE: %.4f\tTest MSE:%f' \
                % (epoch+1, epochs, i, total_steps, loss.item(), test_fn(encoder, decoder, testloader, device))
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print out training progress (on different line):
        if (i % total_steps == 0):
            print('\r' + stats)

[1/30][0/29]	Train MSE: 0.2318	Test MSE:0.088387
[2/30][0/29]	Train MSE: 0.0105	Test MSE:0.0009450
[3/30][0/29]	Train MSE: 0.0173	Test MSE:0.0011995
[4/30][0/29]	Train MSE: 0.0123	Test MSE:0.0007839
[5/30][0/29]	Train MSE: 0.0095	Test MSE:0.0008880
[6/30][0/29]	Train MSE: 0.0054	Test MSE:0.0008018
[7/30][0/29]	Train MSE: 0.0122	Test MSE:0.0007705
[8/30][0/29]	Train MSE: 0.0092	Test MSE:0.0007214
[9/30][0/29]	Train MSE: 0.0206	Test MSE:0.0008764
[10/30][0/29]	Train MSE: 0.0030	Test MSE:0.000965
[11/30][0/29]	Train MSE: 0.0059	Test MSE:0.0009792
[12/30][0/29]	Train MSE: 0.0050	Test MSE:0.0006909
[13/30][0/29]	Train MSE: 0.0118	Test MSE:0.0008567
[14/30][0/29]	Train MSE: 0.0033	Test MSE:0.0021308
[15/30][0/29]	Train MSE: 0.0054	Test MSE:0.0008122
[16/30][0/29]	Train MSE: 0.0047	Test MSE:0.0011070
[17/30][0/29]	Train MSE: 0.0058	Test MSE:0.0007059
[18/30][0/29]	Train MSE: 0.0049	Test MSE:0.0006728
[19/30][0/29]	Train MSE: 0.0056	Test MSE:0.0006210
[20/30][0/29]	Train MSE: 0.0035	Test MSE:0

#####################################################

In [15]:
%load_ext autoreload
%autoreload 2
import os, sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataset import *
from models import *

from sklearn.preprocessing import MinMaxScaler
import mxnet as mx
from mxnet import nd, init, gluon, context
from mxnet.gluon.data import Dataset, ArrayDataset, DataLoader
from multiprocessing import cpu_count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Load and preprocess data:
path = 'stocks/individual_stocks_5yr/'
stocks = os.listdir(path)
# Load file, pick only open, high, low, close, and convert to numpy array:
ts = pd.read_csv(path+stocks[100]).select_dtypes(['float']).values
# Normalize data to range [-1, 1]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ts_norm = scaler.fit_transform(ts)

In [12]:
# Get train loader and test loader:
test_size = 0.2
source_len = 90
target_len = 1
batch_size = 8
trainset, testset, trainloader, testloader = get_datasets_loaders(ts_norm, test_size, source_len, target_len, batch_size)

In [13]:
# Hyper-parameters:
batch_size = 32
output_size = 4
hidden_size = 256
num_layers = 1
dropout = 0
batch_first = True

epochs = 30
lr = 1e-3

total_steps = np.ceil(len(trainset) / batch_size)
print_every = 5

In [14]:
# Initialize model:
model = ManyToOne(output_size, hidden_size, num_layers, dropout)
model.initialize(init=init.Xavier())
# Choose a loss function:
loss = gluon.loss.L2Loss()
# Choose a trainer:
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 1e-2})

In [19]:
devices = [context.gpu(i) for i in range(context.num_gpus())]

In [275]:
# Training loop:
for epoch in range(epochs):
    for i, (src, tgt) in enumerate(trainloader):
        src, tgt = src.as_in_context(mx.gpu(0)), tgt.as_in_context(mx.gpu(0))
        # Forward + backward:
        with autograd.record():
            src, tgt = ctx.to(device), tgt.to(device)
        output = model(ctx)
        loss = F.mse_loss(output, tgt.squeeze())
        loss.backward()
        optimizer.step()
        
        # Print out training progress (on same line):
        stats = '[%d/%d][%d/%d]\tLoss: %.4f' \
                % (epoch+1, epochs, i, total_steps, loss.item())
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print out training progress (on different line):
        if (i % total_steps == 0):
            print('\r' + stats)

AttributeError: 'Tensor' object has no attribute 'as_in_context'